# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Data-Challenge-:-Mangaki---September-2017" data-toc-modified-id="Data-Challenge-:-Mangaki---September-2017-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Challenge : Mangaki - September 2017</a></div><div class="lev2 toc-item"><a href="#Reading-data" data-toc-modified-id="Reading-data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Reading data</a></div><div class="lev2 toc-item"><a href="#First-prediction-model" data-toc-modified-id="First-prediction-model-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>First prediction model</a></div><div class="lev2 toc-item"><a href="#Better-predicted-models" data-toc-modified-id="Better-predicted-models-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Better predicted models</a></div><div class="lev3 toc-item"><a href="#Using-watched.csv" data-toc-modified-id="Using-watched.csv-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Using <code>watched.csv</code></a></div><div class="lev4 toc-item"><a href="#Maping-string-rating-to-probability-of-seeing-the-movie" data-toc-modified-id="Maping-string-rating-to-probability-of-seeing-the-movie-1311"><span class="toc-item-num">1.3.1.1&nbsp;&nbsp;</span>Maping string-rating to probability of seeing the movie</a></div><div class="lev3 toc-item"><a href="#Using-titles.csv" data-toc-modified-id="Using-titles.csv-132"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Using <code>titles.csv</code></a></div><div class="lev2 toc-item"><a href="#Evaluation-from-the-data-challenge-platform" data-toc-modified-id="Evaluation-from-the-data-challenge-platform-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Evaluation from the data challenge platform</a></div>

# Data Challenge : Mangaki - September 2017

> - See [here for more information](http://universityofbigdata.net/competition/5085548788056064?lang=en).
> - Author: [Lilian Besson](http://perso.crans.org/besson/).
> - License: [MIT License](https://lbesson.mit-license.org/).

## Reading data
We have a few CSV files, let start by reading them.

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
!ls -larth *.csv

-rw-r--r-- 1 lilian lilian  350K juin  27 15:10 titles.csv
-rw-r--r-- 1 lilian lilian  3,2M juin  27 15:25 watched.csv
-rw-r--r-- 1 lilian lilian 1010K juin  27 15:34 test.csv
-rw-r--r-- 1 lilian lilian  124K juin  28 17:55 train.csv
-rw-r--r-- 1 lilian lilian  2,4M juin  28 17:57 submission.csv


In [3]:
!cp -vf submission.csv submission.csv.old

'submission.csv' -> 'submission.csv.old'


In [57]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
titles = pd.read_csv("titles.csv")
watched = pd.read_csv("watched.csv")

In [56]:
np.unique(titles.category)

array(['album', 'anime', 'manga'], dtype=object)

Just to check they have correctly been read:

In [58]:
train[:5]
len(train)
min(train['user_id']), max(train['user_id'])
min(train['work_id']), max(train['work_id'])

,user_id,work_id,rating
0,50,4041,0
1,508,1713,0
2,1780,7053,1
3,658,8853,0
4,1003,9401,0


11112

(1, 1982)

(2, 9884)

In [6]:
test[:5]
len(test)
min(test['user_id']), max(test['user_id'])
min(test['work_id']), max(test['work_id'])

,user_id,work_id
0,486,1086
1,1509,3296
2,617,1086
3,270,9648
4,459,3647


100015

(0, 1982)

(2, 9884)

In [7]:
watched[:5]
len(watched)
min(watched['user_id']), max(watched['user_id'])
min(watched['work_id']), max(watched['work_id'])

,user_id,work_id,rating
0,717,8025,dislike
1,1106,1027,neutral
2,1970,3949,neutral
3,1685,9815,like
4,1703,3482,like


198970

(0, 1982)

(0, 9896)

## First prediction model

- For each movie, compute the empirical average `rating` of users who saw it, using data from the train data.
- And simply use this to predict for the other users in test data.

In [32]:
submission = test.copy()

In [33]:
total_average_rating = train.rating.mean()

In [34]:
submission[:5]
len(submission)

,user_id,work_id
0,486,1086
1,1509,3296
2,617,1086
3,270,9648
4,459,3647


100015

In [35]:
works_id = np.unique(np.append(test.work_id.unique(), train.work_id.unique()))

In [36]:
mean_ratings = pd.DataFrame(data={'mean_rating': 0}, index=works_id)
mean_ratings[:5]
len(mean_ratings)

,mean_rating
2,0
3,0
4,0
5,0
9,0


2706

In [37]:
computed_means = pd.DataFrame(data={'mean_rating': train.groupby('work_id').mean()['rating']}, index=works_id)
computed_means[:5]
len(computed_means)

,mean_rating
2,0.230769
3,NaN
4,0.500000
5,0.333333
9,0.200000


2706

In [38]:
mean_ratings.update(computed_means)

In [39]:
mean_ratings[:10]
len(mean_ratings)

,mean_rating
2,0.230769
3,0.000000
4,0.500000
5,0.333333
9,0.200000
22,0.000000
23,0.000000
24,0.000000
27,0.333333
28,0.333333


2706

In [41]:
submission = submission.join(mean_ratings, on='work_id')
submission.rename_axis({'mean_rating': 'prob_willsee'}, axis="columns", inplace=True)

In [43]:
# in case of mean on empty values
submission.fillna(value=total_average_rating, inplace=True)

In [51]:
submission[:10]

,user_id,work_id,prob_willsee
0,486,1086,0.440000
1,1509,3296,0.000000
2,617,1086,0.440000
3,270,9648,0.529412
4,459,3647,0.333333
5,41,3562,0.724138
6,1780,9156,0.333333
7,284,5502,0.500000
8,1521,7250,0.500000
9,130,8209,0.500000


Let save it to `submission_naive1.csv`:

In [52]:
submission.to_csv("submission_naive1.csv", index=False)

In [49]:
!ls -larth submission_naive1.csv

-rw-rw-r-- 1 lilian lilian 2,0M sept. 24 14:40 submission_naive1.csv


## Better predicted models

### Using `watched.csv`

The bonus data set `watched` can give a lot of information. There is 200000 entries in it and only 100000 in `test.csv`.

In [66]:
len(test), len(watched)

(100015, 198970)

In [68]:
ratings = np.unique(watched.rating).tolist()
ratings

['dislike', 'like', 'love', 'neutral']

In [67]:
watched[:5]

,user_id,work_id,rating
0,717,8025,dislike
1,1106,1027,neutral
2,1970,3949,neutral
3,1685,9815,like
4,1703,3482,like


#### Maping string-rating to probability of seeing the movie
By using the train data `(user, work)` that are also in `watched`, we can learn to map string rating, i.e., `'dislike', 'neutral', 'like', 'love'`, to probability of having see the movie.

In [80]:
mapping_strrating_probwillsee = {
    'dislike': 0,
    'neutral': 0.50,
    'like': 0.75,
    'love': 1,
}

In [81]:
watched[:5]

,user_id,work_id,rating
0,717,8025,dislike
1,1106,1027,neutral
2,1970,3949,neutral
3,1685,9815,like
4,1703,3482,like


In [69]:
train[:5]

,user_id,work_id,rating
0,50,4041,0
1,508,1713,0
2,1780,7053,1
3,658,8853,0
4,1003,9401,0


In [79]:
watched[(watched.user_id == 50) & (watched.work_id == 1713)]

,user_id,work_id,rating


### Using `titles.csv`
I don't think I want to use the titles, but clustering the works by categories could help, maybe.

In [63]:
categories = np.unique(titles.category).tolist()
categories

['album', 'anime', 'manga']

In [64]:
for cat in categories:
    print("There is {} works in category {}".format(sum(titles.category == cat), cat))

There is 1 works in category album
There is 7088 works in category anime
There is 2808 works in category manga


One category is alone, let rewrite it to `'anime'`.

In [65]:
categories = {
    'anime': 0,
    'album': 0,
    'manga': 1,
}

## Evaluation from the data challenge platform

TODO !